In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
import pandas as pd
os.chdir("..")

In [2]:
from recoexplainer.config import cfg
from recoexplainer.data_reader.data_reader import DataReader
from recoexplainer.models.emf_model import EMFModel

In [3]:
data = DataReader(cfg.ml100k)
data.make_consecutive_ids_in_dataset()

In [4]:
emf = EMFModel(cfg.model.emf)

In [5]:
emf.fit(data)

Range of userId is [0, 942]
Range of itemId is [0, 1681]


True

In [6]:
mf = EMFModel(cfg.model.mf)

In [7]:
mf.fit(data)

Range of userId is [0, 942]
Range of itemId is [0, 1681]


True

In [9]:
from recoexplainer.recommender import RankPredictionsRecommender
rec = RankPredictionsRecommender(data, emf)

In [17]:
rec.recommend_all()

,userId,itemId,rank
1561,0.0,1653.0,1.0
1549,0.0,1641.0,2.0
796,0.0,875.0,3.0
1240,0.0,1331.0,4.0
1428,0.0,1520.0,5.0
...,...,...,...
12,942.0,14.0,6.0
328,942.0,342.0,7.0
443,942.0,459.0,8.0
1074,942.0,1096.0,9.0


In [10]:
data.dataset

,userId,itemId,rating,timestamp
0,0,0,3,891717742
1,1,1,1,878887116
2,2,2,2,880606923
3,3,3,1,886397596
4,4,4,4,884182806
...,...,...,...,...
99994,875,172,3,880175444
99995,708,247,5,879795543
99996,36,1004,1,874795795
99997,57,443,2,882399156
